In [ ]:
import pickle 
import pandas as pd 
import os
import numpy as np 
import cv2
from keras.preprocessing import image 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import Functions as func
import var as var

In [ ]:
#negatives 
df = pd.DataFrame(columns = ['path', 'label'])
negative = [f'../FinalImages/Negative/{i}' for i in os.listdir('../FinalImages/Negative')]
neg_labels = [0 for i in range(len(negative))]
positive = [f'../FinalImages/Positive/{i}' for i in os.listdir('../FinalImages/Positive')]
pos_labels = [1 for i in range(len(positive))]
concat_path = negative[:len(positive)+ len(positive)] + positive
concat_labels = neg_labels[:len(positive)+ len(positive)] + pos_labels


df.path = concat_path 
df.label = concat_labels 

X = df
y = df[['label']]

In [ ]:
df.head(2)
print(df.label.value_counts())

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify = X.label, train_size = .95, random_state = 10)

x_train.drop('label', axis = 1, inplace = True)
x_test.drop('label', axis = 1, inplace = True)

In [ ]:
def get_img_array(img_paths, dim, img_type): 
    final_array = []
    for path in tqdm(img_paths): 
        img = func.get_image_value(path, dim, img_type)
        final_array.append(img)
    if img_type == 'grey':
        final_array = np.array(final_array)
        return final_array.reshape(final_array.shape[0], var.dimension[0],var.dimension[0],1)
    else: 
        return np.array(final_array)

new_x_train = get_img_array(x_train.path.values, var.mobilenet_dimension, img_type = var.img_type)
new_x_test = get_img_array(x_test.path.values, var.mobilenet_dimension, img_type = var.img_type)

In [ ]:
new_x_train.shape

In [ ]:
# def pickle_large_file(value, file_path): 
    
#     n_bytes = 2**31
#     max_bytes = 2**31 - 1
#     data = bytearray(n_bytes)

#     ## write
#     bytes_out = pickle.dump(data)
#     with open(file_path, 'wb') as f_out:
#         for idx in range(0, len(bytes_out), max_bytes):
#             f_out.write(bytes_out[idx:idx+max_bytes])

#     ## read
#     bytes_in = bytearray(0)
#     input_size = os.path.getsize(file_path)
#     with open(file_path, 'rb') as f_in:
#         for _ in range(0, input_size, max_bytes):
#             bytes_in += f_in.read(max_bytes)
#     data2 = pickle.loads(bytes_in)

#     assert(data == data2)

In [ ]:
#save pickles

pickle.dump(new_x_train, open('../Pickles/mobile_net_x_train.p', 'wb'), protocol=4)
pickle.dump(y_train, open('../Pickles/mobile_net_y_train.p', 'wb'), protocol=4)
pickle.dump(new_x_test, open('../Pickles/mobile_net_x_test.p', 'wb'), protocol=4)
pickle.dump(y_test, open('../Pickles/mobile_net_y_test.p', 'wb'), protocol=4)

In [ ]:
assert False

In [ ]:
y_train.values
cv2.imshow('test', new_x_train[2])
cv2.waitKey(0)
cv2.destroyAllWindows()